# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
#     print(source_vocab_to_int)
#     print([word for word in source_text.split()])
    source_int = [[source_vocab_to_int[word] for word in se.split()] for se in source_text.split('\n')]
    target_int = [[target_vocab_to_int[word] for word in se.split()+['<EOS>']] for se in target_text.split('\n')]
    return source_int, target_int

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob= tf.placeholder(tf.float32,name="keep_prob")

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, keep_prob,target_sequence_length, max_target_sequence_length, source_sequence_length



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    embedding=tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    def make_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-.1,0.1,seed=3))
        dropout=tf.contrib.rnn.DropoutWrapper(cell,input_keep_prob=keep_prob)
        return dropout
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, embedding, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):

        # Helper for the training process. Used by BasicDecoder to read inputs.
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
        
        # Basic decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        
        # Perform dynamic decoding using the decoder
        training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_summary_length)[0]
    

    return training_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=None):
        start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
        # Perform dynamic decoding using the decoder
        inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
         
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size, i_layer):
        with tf.variable_scope('decoder_{}'.format(i_layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                               initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
            return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, i) for i in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    with tf.variable_scope("decoding") as decoding_scope:
        output_layer = Dense(target_vocab_size,
                             kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1),
                             name='output_layer')

        output= decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, 
                                     max_target_sequence_length, output_layer, keep_prob)

        infer = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                                     target_vocab_to_int['<EOS>'], max_target_sequence_length, target_vocab_size, 
                                     output_layer, batch_size, keep_prob)

    return output, infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

(64, ?, 1000)
[64, None, 1000]
Tests Passed


In [13]:
%load_ext autoreload
%autoreload 2

### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
     # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length=source_sequence_length,
                                  source_vocab_size=source_vocab_size, 
                                  encoding_embedding_size=enc_embedding_size,
                                  keep_prob=keep_prob)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output =  decoding_layer(dec_input, enc_state, target_sequence_length, 
                                                                        max_target_sentence_length,
                                                                        rnn_size, num_layers, target_vocab_to_int, 
                                                                        target_vocab_size, batch_size, keep_prob, 
                                                                        dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 3
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 200
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 300
decoding_embedding_size = 300
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.8
display_step = 2

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/1077 - Train Accuracy: 0.2463, Validation Accuracy: 0.3363, Loss: 4.1613
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2641, Validation Accuracy: 0.3398, Loss: 3.6065
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2938, Validation Accuracy: 0.3498, Loss: 3.4253
Epoch   0 Batch    8/1077 - Train Accuracy: 0.2719, Validation Accuracy: 0.3409, Loss: 3.3189
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2870, Validation Accuracy: 0.3718, Loss: 3.3033
Epoch   0 Batch   12/1077 - Train Accuracy: 0.3164, Validation Accuracy: 0.3746, Loss: 3.0886
Epoch   0 Batch   14/1077 - Train Accuracy: 0.3661, Validation Accuracy: 0.4034, Loss: 2.8617
Epoch   0 Batch   16/1077 - Train Accuracy: 0.3797, Validation Accuracy: 0.4180, Loss: 2.8409
Epoch   0 Batch   18/1077 - Train Accuracy: 0.3715, Validation Accuracy: 0.4329, Loss: 2.8017
Epoch   0 Batch   20/1077 - Train Accuracy: 0.3852, Validation Accuracy: 0.4311, Loss: 2.6622
Epoch   0 Batch   22/1077 - Train Accuracy: 0.3992, Validati

Epoch   0 Batch  178/1077 - Train Accuracy: 0.6492, Validation Accuracy: 0.6186, Loss: 0.5924
Epoch   0 Batch  180/1077 - Train Accuracy: 0.6340, Validation Accuracy: 0.5902, Loss: 0.6046
Epoch   0 Batch  182/1077 - Train Accuracy: 0.6484, Validation Accuracy: 0.6030, Loss: 0.5862
Epoch   0 Batch  184/1077 - Train Accuracy: 0.6656, Validation Accuracy: 0.6065, Loss: 0.5752
Epoch   0 Batch  186/1077 - Train Accuracy: 0.6402, Validation Accuracy: 0.6349, Loss: 0.5867
Epoch   0 Batch  188/1077 - Train Accuracy: 0.6410, Validation Accuracy: 0.6531, Loss: 0.5851
Epoch   0 Batch  190/1077 - Train Accuracy: 0.6750, Validation Accuracy: 0.6243, Loss: 0.5625
Epoch   0 Batch  192/1077 - Train Accuracy: 0.6570, Validation Accuracy: 0.6044, Loss: 0.5609
Epoch   0 Batch  194/1077 - Train Accuracy: 0.6819, Validation Accuracy: 0.6317, Loss: 0.5276
Epoch   0 Batch  196/1077 - Train Accuracy: 0.6836, Validation Accuracy: 0.6516, Loss: 0.5405
Epoch   0 Batch  198/1077 - Train Accuracy: 0.7254, Validati

Epoch   0 Batch  354/1077 - Train Accuracy: 0.8285, Validation Accuracy: 0.8050, Loss: 0.2897
Epoch   0 Batch  356/1077 - Train Accuracy: 0.8227, Validation Accuracy: 0.8171, Loss: 0.2554
Epoch   0 Batch  358/1077 - Train Accuracy: 0.7915, Validation Accuracy: 0.8050, Loss: 0.2841
Epoch   0 Batch  360/1077 - Train Accuracy: 0.8172, Validation Accuracy: 0.7940, Loss: 0.2424
Epoch   0 Batch  362/1077 - Train Accuracy: 0.7995, Validation Accuracy: 0.7958, Loss: 0.2607
Epoch   0 Batch  364/1077 - Train Accuracy: 0.8094, Validation Accuracy: 0.8086, Loss: 0.2709
Epoch   0 Batch  366/1077 - Train Accuracy: 0.8156, Validation Accuracy: 0.7972, Loss: 0.2355
Epoch   0 Batch  368/1077 - Train Accuracy: 0.8449, Validation Accuracy: 0.7965, Loss: 0.2508
Epoch   0 Batch  370/1077 - Train Accuracy: 0.8683, Validation Accuracy: 0.7937, Loss: 0.2392
Epoch   0 Batch  372/1077 - Train Accuracy: 0.8480, Validation Accuracy: 0.8047, Loss: 0.2149
Epoch   0 Batch  374/1077 - Train Accuracy: 0.8227, Validati

Epoch   0 Batch  530/1077 - Train Accuracy: 0.8676, Validation Accuracy: 0.8409, Loss: 0.1228
Epoch   0 Batch  532/1077 - Train Accuracy: 0.8688, Validation Accuracy: 0.8274, Loss: 0.1236
Epoch   0 Batch  534/1077 - Train Accuracy: 0.8798, Validation Accuracy: 0.8388, Loss: 0.1228
Epoch   0 Batch  536/1077 - Train Accuracy: 0.8594, Validation Accuracy: 0.8658, Loss: 0.1135
Epoch   0 Batch  538/1077 - Train Accuracy: 0.9215, Validation Accuracy: 0.8675, Loss: 0.0923
Epoch   0 Batch  540/1077 - Train Accuracy: 0.8926, Validation Accuracy: 0.8633, Loss: 0.0999
Epoch   0 Batch  542/1077 - Train Accuracy: 0.8926, Validation Accuracy: 0.8565, Loss: 0.1046
Epoch   0 Batch  544/1077 - Train Accuracy: 0.9031, Validation Accuracy: 0.8771, Loss: 0.0918
Epoch   0 Batch  546/1077 - Train Accuracy: 0.8785, Validation Accuracy: 0.8693, Loss: 0.1305
Epoch   0 Batch  548/1077 - Train Accuracy: 0.8965, Validation Accuracy: 0.8867, Loss: 0.1228
Epoch   0 Batch  550/1077 - Train Accuracy: 0.8293, Validati

Epoch   0 Batch  706/1077 - Train Accuracy: 0.8560, Validation Accuracy: 0.8803, Loss: 0.1124
Epoch   0 Batch  708/1077 - Train Accuracy: 0.9020, Validation Accuracy: 0.8928, Loss: 0.0853
Epoch   0 Batch  710/1077 - Train Accuracy: 0.9184, Validation Accuracy: 0.8984, Loss: 0.0648
Epoch   0 Batch  712/1077 - Train Accuracy: 0.9238, Validation Accuracy: 0.8881, Loss: 0.0667
Epoch   0 Batch  714/1077 - Train Accuracy: 0.8783, Validation Accuracy: 0.8814, Loss: 0.0872
Epoch   0 Batch  716/1077 - Train Accuracy: 0.9168, Validation Accuracy: 0.9009, Loss: 0.0655
Epoch   0 Batch  718/1077 - Train Accuracy: 0.9094, Validation Accuracy: 0.8746, Loss: 0.0745
Epoch   0 Batch  720/1077 - Train Accuracy: 0.9264, Validation Accuracy: 0.8757, Loss: 0.0893
Epoch   0 Batch  722/1077 - Train Accuracy: 0.9051, Validation Accuracy: 0.9070, Loss: 0.0638
Epoch   0 Batch  724/1077 - Train Accuracy: 0.9211, Validation Accuracy: 0.8881, Loss: 0.0797
Epoch   0 Batch  726/1077 - Train Accuracy: 0.9230, Validati

...

Epoch   2 Batch  318/1077 - Train Accuracy: 0.9609, Validation Accuracy: 0.9641, Loss: 0.0316
Epoch   2 Batch  320/1077 - Train Accuracy: 0.9441, Validation Accuracy: 0.9592, Loss: 0.0416
Epoch   2 Batch  322/1077 - Train Accuracy: 0.9345, Validation Accuracy: 0.9549, Loss: 0.0389
Epoch   2 Batch  324/1077 - Train Accuracy: 0.9340, Validation Accuracy: 0.9503, Loss: 0.0322
Epoch   2 Batch  326/1077 - Train Accuracy: 0.9721, Validation Accuracy: 0.9506, Loss: 0.0392
Epoch   2 Batch  328/1077 - Train Accuracy: 0.9278, Validation Accuracy: 0.9389, Loss: 0.0457
Epoch   2 Batch  330/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9308, Loss: 0.0450
Epoch   2 Batch  332/1077 - Train Accuracy: 0.9706, Validation Accuracy: 0.9247, Loss: 0.0271
Epoch   2 Batch  334/1077 - Train Accuracy: 0.9527, Validation Accuracy: 0.9247, Loss: 0.0386
Epoch   2 Batch  336/1077 - Train Accuracy: 0.9215, Validation Accuracy: 0.9322, Loss: 0.0581
Epoch   2 Batch  338/1077 - Train Accuracy: 0.9070, Validati

Epoch   2 Batch  494/1077 - Train Accuracy: 0.9383, Validation Accuracy: 0.9201, Loss: 0.0375
Epoch   2 Batch  496/1077 - Train Accuracy: 0.9551, Validation Accuracy: 0.9286, Loss: 0.0425
Epoch   2 Batch  498/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9336, Loss: 0.0460
Epoch   2 Batch  500/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9308, Loss: 0.0305
Epoch   2 Batch  502/1077 - Train Accuracy: 0.9395, Validation Accuracy: 0.9247, Loss: 0.0469
Epoch   2 Batch  504/1077 - Train Accuracy: 0.9484, Validation Accuracy: 0.9286, Loss: 0.0329
Epoch   2 Batch  506/1077 - Train Accuracy: 0.9406, Validation Accuracy: 0.9190, Loss: 0.0506
Epoch   2 Batch  508/1077 - Train Accuracy: 0.9650, Validation Accuracy: 0.9244, Loss: 0.0318
Epoch   2 Batch  510/1077 - Train Accuracy: 0.9258, Validation Accuracy: 0.9332, Loss: 0.0409
Epoch   2 Batch  512/1077 - Train Accuracy: 0.9668, Validation Accuracy: 0.9233, Loss: 0.0281
Epoch   2 Batch  514/1077 - Train Accuracy: 0.9410, Validati

Epoch   2 Batch  670/1077 - Train Accuracy: 0.9513, Validation Accuracy: 0.9467, Loss: 0.0417
Epoch   2 Batch  672/1077 - Train Accuracy: 0.9539, Validation Accuracy: 0.9332, Loss: 0.0277
Epoch   2 Batch  674/1077 - Train Accuracy: 0.9703, Validation Accuracy: 0.9375, Loss: 0.0357
Epoch   2 Batch  676/1077 - Train Accuracy: 0.9416, Validation Accuracy: 0.9347, Loss: 0.0362
Epoch   2 Batch  678/1077 - Train Accuracy: 0.9435, Validation Accuracy: 0.9279, Loss: 0.0332
Epoch   2 Batch  680/1077 - Train Accuracy: 0.9289, Validation Accuracy: 0.9332, Loss: 0.0337
Epoch   2 Batch  682/1077 - Train Accuracy: 0.9379, Validation Accuracy: 0.9308, Loss: 0.0421
Epoch   2 Batch  684/1077 - Train Accuracy: 0.9512, Validation Accuracy: 0.9350, Loss: 0.0413
Epoch   2 Batch  686/1077 - Train Accuracy: 0.9635, Validation Accuracy: 0.9567, Loss: 0.0324
Epoch   2 Batch  688/1077 - Train Accuracy: 0.9582, Validation Accuracy: 0.9535, Loss: 0.0344
Epoch   2 Batch  690/1077 - Train Accuracy: 0.9289, Validati

Epoch   2 Batch  846/1077 - Train Accuracy: 0.9426, Validation Accuracy: 0.9659, Loss: 0.0477
Epoch   2 Batch  848/1077 - Train Accuracy: 0.9555, Validation Accuracy: 0.9616, Loss: 0.0327
Epoch   2 Batch  850/1077 - Train Accuracy: 0.9353, Validation Accuracy: 0.9545, Loss: 0.0724
Epoch   2 Batch  852/1077 - Train Accuracy: 0.9070, Validation Accuracy: 0.9638, Loss: 0.0500
Epoch   2 Batch  854/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9634, Loss: 0.0434
Epoch   2 Batch  856/1077 - Train Accuracy: 0.9246, Validation Accuracy: 0.9560, Loss: 0.0396
Epoch   2 Batch  858/1077 - Train Accuracy: 0.9617, Validation Accuracy: 0.9556, Loss: 0.0319
Epoch   2 Batch  860/1077 - Train Accuracy: 0.9379, Validation Accuracy: 0.9492, Loss: 0.0406
Epoch   2 Batch  862/1077 - Train Accuracy: 0.9477, Validation Accuracy: 0.9538, Loss: 0.0464
Epoch   2 Batch  864/1077 - Train Accuracy: 0.9320, Validation Accuracy: 0.9435, Loss: 0.0395
Epoch   2 Batch  866/1077 - Train Accuracy: 0.9282, Validati

Epoch   2 Batch 1022/1077 - Train Accuracy: 0.9706, Validation Accuracy: 0.9396, Loss: 0.0364
Epoch   2 Batch 1024/1077 - Train Accuracy: 0.9266, Validation Accuracy: 0.9418, Loss: 0.0470
Epoch   2 Batch 1026/1077 - Train Accuracy: 0.9866, Validation Accuracy: 0.9442, Loss: 0.0374
Epoch   2 Batch 1028/1077 - Train Accuracy: 0.9345, Validation Accuracy: 0.9240, Loss: 0.0359
Epoch   2 Batch 1030/1077 - Train Accuracy: 0.9617, Validation Accuracy: 0.9311, Loss: 0.0379
Epoch   2 Batch 1032/1077 - Train Accuracy: 0.9293, Validation Accuracy: 0.9272, Loss: 0.0475
Epoch   2 Batch 1034/1077 - Train Accuracy: 0.9520, Validation Accuracy: 0.9389, Loss: 0.0430
Epoch   2 Batch 1036/1077 - Train Accuracy: 0.9263, Validation Accuracy: 0.9354, Loss: 0.0480
Epoch   2 Batch 1038/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9361, Loss: 0.0490
Epoch   2 Batch 1040/1077 - Train Accuracy: 0.9465, Validation Accuracy: 0.9396, Loss: 0.0403
Epoch   2 Batch 1042/1077 - Train Accuracy: 0.9480, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    lower = sentence.lower()
    words =lower.split()
    
    return [vocab_to_int[w] if w in vocab_to_int else vocab_to_int['<UNK>'] for w in words]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [114, 181, 77, 23, 99, 178, 141]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [193, 218, 113, 219, 195, 100, 258, 153, 1]
  French Words: il a l' se rendre voiture jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.